---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [99]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df


0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [157]:
def date_sorter():
    import numpy as np
    months={'Jan': '01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 
           'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
    
    c1=df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})')
    c1['year']=c1['year'].apply(lambda x: '19'+ x if len(x) == 2 else x)
    c1['month']=c1['month'].apply(lambda x: '0' + x if len(x)==1 else x)
    c1['day']=c1['day'].apply(lambda x: '0' + x if len(x)==1 else x)

    c2=df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.]?[- ](?P<day>\d{1,2})[,]?[- ](?P<year>\d{4})')
    c2=c2.replace({'month':months})
    
    c3=df.str.extractall(r'(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]? (?P<year>\d{4})')
    c3=c3.replace({'month':months})
    
    c4=df.str.extractall(r'(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})')
    c4=c4[['month', 'day', 'year']]
    c4['month']=c4['month'].apply(lambda x: x[:3] if len(x) > 3 else x)
    c4=c4.replace({'month':months})
    c4['month']=c4['month'].apply(lambda x: '01' if x=='Age' else x)
    c5=df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[,]? (?P<year>\d{4})')
    c5=c5.replace({'month':months})
    c5['day']='01'
    
    
    c6=df.str.extractall(r'(?P<month>\d{1,2})[/](?P<year>\d{4})')
    c6['day']='01'
    c6=c6.replace({'month':months})

    c7=df.str.extractall(r'(?P<year>\d{4})+[^0-9]+')
    c7['year']=c7['year'].apply(lambda x: x if x.startswith('19') or x.startswith('20') else 0)
    c7=c7[c7['year']!=0]
    c7['month']= '01'
    c7['day']= '01'
    
    
    c1=c1.reset_index()
    c2=c2.reset_index()
    c3=c3.reset_index()
    c4=c4.reset_index()
    c5=c5.reset_index()
    c6=c6.reset_index()
    c7=c7.reset_index()

    
    df1=pd.concat([c1,c2,c3,c4,c5,c6,c7]).drop_duplicates(subset='level_0', keep='first').reset_index(drop=True)   
    
    df1['month']=df1['month'].apply(lambda x: str(x))
    df1['day']=df1['day'].apply(lambda x: str(x))
    df1['day']=df1['day'].apply(lambda x: '0' + x if len(x)==1 else x)
    df1['month']=df1['month'].apply(lambda x: '0' + x if len(x)==1 else x)
    
    df1['date']=df1['year']+df1['month']+df1['day']
    df_sorted=df1.sort_values(by=['date'])
    df_sorted=df_sorted.reset_index()
    
    return pd.Series(df_sorted['level_0'])
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
      ... 
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64

###### 